## Web scraping(웹 스크래핑, 문서 수집하기)
[Open In Colab](https://colab.research.google.com/github/langchain-ai/langchain/blob/v0.1/docs/docs/use_cases/web_scraping.ipynb)

웹 리서치는 대표적인 LLM 응용 프로그램
- 사용자들이 가장 관심가지는 분야중의 하나
- LLM의 한계점인 최신 정보를 가져올 수 있음
- [Gpt-researcher](https://github.com/assafelovic/gpt-researcher) 인기 상승  
<img src="./images/gpt-researcher.jpg" width="600"> <img src="./images/gpt-researcher_architecture.png" width="398">


### 구성도
![](./images/web_scraping.png)


### 주요 컴포넌트
- Searcher(검색)
  - 1. 쿼리 생성 (유의어를 포함한 자동 확장)
  - 2. URL 생성
  - ex) GoogleSearchAPIWrapper
- Loader(수집)
  - URL로 문서 가져오기
  - [AsyncHtmlLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.async_html.AsyncHtmlLoader.html)
    - [Document loaders](https://python.langchain.com/v0.2/docs/integrations/document_loaders/)
    - aiohttp 라이브러리를 사용하여 비동기 HTTP로 HTML 문서를 수집
    - 단순한 웹사이트 자료를 가져오는데 적합
  - [AsyncChromiumLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.chromium.AsyncChromiumLoader.html)
    - Playwright를 사용하여 Chromium 인스턴스를 실행하여 HTML 문서를 가져옴
    - 인증이 있거나(카페 자료) 자바스크립트 등의 복잡한 사이트(쇼핑몰)에 적합
    - 카페의 자료는 상대적으로 주제가 한정되고, 고품질의 데이터가 많이 있음
    - Chromium은 Playwright에서 지원하는 브라우저 중 하나이며, chromium, ffmpeg, firefox, webkit등이 있음
    - 헤드리스 모드를 사용하면 브라우저가 실행되지 않아서 속도가 빠름
- Transformer(변환)
  - HTML 코드 전처리
  - HTML2Text
    - 태그를 제외한 텍스트 추출
  - BeautifulSoup
    - 태그를 DOM Tree로 변환한후 텍스트 추출
    - 사용자가 원하는 내용만 정확하게 추출할 수 있음

### Scraper 주의 사항
- 크롤러 vs 스크래퍼
  - 크롤러: 특정 웹 사이트 내 '모든' 하이퍼링크를 순회하면서 자료 수집
  - 스크래퍼: 웹 페이지에서 '필요한' 부분만 수집
- 짧은 시간 동안 서버에 자료를 요청
  - 목적지 서버에 부하를 가중시킬 수 있음
  - 출발지 IP 차단 가능성
    - 차단될 경우 해제 요청을 수동으로 진행하거나 일정 시간이 지나야 다시 접근할 수 있음
    - 출발지가 차단될 경우 공인 IP가 차단되므로 같은 공간의 모든 네트워크가 차단될 수 있음

### Resources
 - [랭체인(langchain) + 웹사이트 정보 추출 - 스키마 활용법 (6)](https://teddylee777.github.io/langchain/langchain-tutorial-06/)
 - [LangChain > Use cases > More > Web scraping](https://python.langchain.com/v0.1/docs/use_cases/web_scraping/)


### 데모 순서
#### 1. Loader(AsyncHtmlLoader, AsyncChromiumLoader)
#### 2. Transformer(Html2Text, Beautifulsoup)
#### 3. Scraping with extraction
#### 4. Research automation

In [1]:
from python_environment_check import check_packages

lib = {
    'langchain-openai': '0.1.7',
    'langchain-core': '0.2.0',
    'langchain': '0.0.354',
    'langchain_text_splitters': '0.2.0',
    'playwright': '1.44.0',
    'beautifulsoup4': '4.12.2',
    'html2text': '2024.2.26',
    'dotenv': '0.0.5',
    'nest_asyncio': '1.6.0',
}

#check_packages(lib)

[OK] Your Python version is 3.11.9 (main, May 28 2024, 08:05:11) [GCC 8.5.0 20210514 (Red Hat 8.5.0-20)]


In [8]:
import os
import dotenv
dotenv.load_dotenv('envls')

# 주피터 노트북에서 사용할 경우 미리 실행해야 함
import nest_asyncio
nest_asyncio.apply()



# 미리 설치해야 함
# pip install playwright
# !playwright install
# Rocky8
# Chromium
sudo dnf update -y && \
    dnf install -y alsa-lib \
    at-spi2-atk \
    at-spi2-core \
    atk \
    bash \
    cairo \
    cups-libs \
    dbus-libs \
    expat \
    flac-libs \
    gdk-pixbuf2 \
    glib2 \
    glibc \
    gtk3 \
    libX11 \
    libXcomposite \
    libXdamage \
    libXext \
    libXfixes \
    libXrandr \
    libXtst \
    libcanberra-gtk3 \
    libdrm \
    libgcc \
    libstdc++ \
    libxcb \
    libxkbcommon \
    libxshmfence \
    libxslt \
    mesa-libgbm \
    nspr \
    nss \
    nss-util \
    pango \
    policycoreutils \
    policycoreutils-python-utils \
    zlib 

# firefox
    sudo dnf install -y atk \
    bash \
    cairo \
    cairo-gobject \
    dbus-glib \
    dbus-libs \
    fontconfig \
    freetype \
    gdk-pixbuf2 \
    glib2 \
    glibc \
    gtk2 \
    gtk3 \
    libX11 \
    libX11-xcb \
    libXcomposite \
    libXcursor \
    libXdamage \
    libXext \
    libXfixes \
    libXi \
    libXrender \
    libXt \
    liberation-fonts-common \
    liberation-sans-fonts \
    libffi \
    libgcc \
    libstdc++ \
    libxcb \
    mozilla-filesystem \
    nspr \
    nss \
    nss-util \
    p11-kit-trust \
    pango \
    pipewire-libs \
    zlib 

# Webkit dependency
sudo dnf install -y harfbuzz-icu \
    libglvnd-glx \
    libglvnd-egl \
    libnotify \
    opus \
    woff2 \
    gstreamer1-plugins-base \
    gstreamer1-plugins-bad-free \
    openjpeg2 \
    libwebp \
    enchant \
    libsecret \
    hyphen \
    libglvnd-gles


In [18]:
# Sample url
sample_url = ["https://www.korea.ac.kr/user/boardList.do?boardId=134&siteId=university&page=1&id=university_060103000000&boardSeq=498426&command=albumView"]

### Loader(AsyncHtmlLoader, AsyncChromiumLoader)
#### 샘플 사이트
<img src="./images/sample_site.png" width="800">

In [19]:
from langchain_community.document_loaders import AsyncHtmlLoader

asynchtml_loader = AsyncHtmlLoader(sample_url)
html = asynchtml_loader.load()

print(type(html), html[0].page_content[:500])

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  1.41it/s]


<class 'list'> 











	













<!doctype html>
<html lang="ko">

<head>
    <meta charset="utf-8" />
    
    <title>고대소식|고대뉴스|행사 · 이벤트</title>
    

    <script type="text/javascript" src="/mbshome/mbs/university/js/jquery-1.9.1.min.js"></script>
    <script type="text/javascript" src="/mbshome/mbs/university/js/jquery-ui.js"></script>
    <!-- <script type="text/javascript" src="/mbshome/mbs/university/js/jquery.easing.1.3.min.js"></script> -->
    <script type="text/javascript" sr


In [20]:
from langchain_community.document_loaders import AsyncChromiumLoader

loader = AsyncChromiumLoader(sample_url)
html = loader.load()

print(type(html), html[0].page_content[:500])

<class 'list'> <!DOCTYPE html><html lang="ko"><head>
    <meta charset="utf-8">
    
    <title>고대소식|고대뉴스|행사 · 이벤트</title>
    

    <script type="text/javascript" async="" src="https://www.google-analytics.com/analytics.js"></script><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/js?id=G-DMLBB3K4EV&amp;cx=c&amp;_slc=1"></script><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/js?id=UA-101799370&amp;l=dataLayer&amp;cx=c"></script><script asyn


### Transformer(Html2Text, Beautifulsoup)
#### 샘플 사이트
<img src="./images/sample_site.png" width="800">

In [22]:
from langchain_community.document_transformers import Html2TextTransformer

html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(html)
docs_transformed[0].page_content[0:500]

'  * 세종캠퍼스\n  * 의료원\n  * KUPID\n  * 발전기금\n  * 120주년 기념 사업\n\n  * Group Service\n    * 예비고대인\n    * 학생·학부모·교직원\n    * 일반인·교우\n    * 장애인\n  * 로그인\n  * ENG\n\n검색창 열기\n\n검색어 입력 go\n\n# 고려대학교\n\n  * 고대소개\n\n## 고대소개\n\nKorea University Since 1905\n\n    * 대학현황\n\n      * 현황\n      * 조직도\n      * 윤리헌장\n      * 학교규칙\n      * 법인\n\n    * 총장실\n\n      * 총장 인사말\n      * Speeches\n      * 총장약력\n      * 총장동정\n      * 역대총장\n\n    * 고대비전\n\n      * 교육목표\n      * 120주년 기념사업\n\n    * 상징\n\n      * UI\n      * 단과대학/대학원 상징\n      * 고대의 상징물\n      * 마스코트\n      * 캐릭터\n'

In [23]:
# print(html)
from langchain_community.document_transformers import BeautifulSoupTransformer

bs_transformer = BeautifulSoupTransformer()
docs_transformed = bs_transformer.transform_documents(html, tags_to_extract=["title"])

# Result
docs_transformed[0].page_content[:500]

'고대소식|고대뉴스|행사 · 이벤트'

---

### 3. Scraping with extraction (LLM을 사용하여 스크래핑)
- 웹 스크래핑 작업이 쉽지 않은 이유
  - 목적지 사이트의 레이아웃이나 콘텐츠가 변경될 수 있음
  - 기존 스크래퍼는 HTML 코드 일부만 변경되어도 프로그램이 정상적으로 동작하지 않음
  - LLM을 사용하면 문맥을 통해 추출하기 때문에 기존 스크래퍼 한계를 보완할 수 있다.

#### 샘플 사이트
<img src="./images/sample_naver_news01.png" width="605"> <img src="./images/sample_naver_news02.png" width="440">

In [28]:
#from langchain_openai import ChatOpenAI # langchain==0.1
from langchain_openai.chat_models.base import ChatOpenAI # langchain==0.2

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")


ImportError: cannot import name 'LangSmithParams' from 'langchain_core.language_models.chat_models' (/home/www/.pyenv/versions/3.11.9/envs/langchain/lib/python3.11/site-packages/langchain_core/language_models/chat_models.py)

In [131]:
from langchain.chains import create_extraction_chain

schema = {
    "properties": {
        "뉴스 출처": {"type": "string"},
        "뉴스 제목": {"type": "string"},
        "뉴스 요약": {"type": "string"},
        "뉴스 URL" : {"type": "string"},
    },
    "required": ["뉴스 출처", "뉴스 제목", "뉴스 요약", "뉴스 URL"],
    #"required": ["뉴스 출처", "뉴스 제목"],
}


def extract(content: str, schema: dict):
    return create_extraction_chain(schema=schema, llm=llm).run(content)

In [132]:
import pprint

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from bs4 import BeautifulSoup

def scrape_with_playwright(urls, schema):
    loader = AsyncChromiumLoader(urls)
    docs = loader.load()

    soup = BeautifulSoup(docs[0].page_content, 'html.parser')
    #docs[0].page_content = soup.select('#main_pack > section > div.api_subject_bx > div.group_news > ul')
    article_tags = soup.select('a.news_tit, a.dsc_txt_wrap, a.press')

    #print(newd)  
    article_text = [a.text + ' ' + a['href'] for a in article_tags]
    
    #print(article_text[:10])
    #[x.text for x in newd.find_all("li")]

    article_docs = []
    article_docs.append(Document(page_content=" ".join(article_text), metadata={"source": urls}))
    print(len(article_docs[0].page_content))
    
    #print(type(fdocs), fdocs)
    """
    bs_transformer = BeautifulSoupTransformer()
    docs_transformed = bs_transformer.transform_documents(
        docs, tags_to_extract=["a", "span"]
    )
    """

    print(article_docs)
    
    #docs_transformed = [x.page_content for x in fdocs]
    #print(type(docs_transformed))
    print("Extracting content with LLM")
    
    # Grab the first 1000 tokens of the site
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=3500, chunk_overlap=0
    )
    splits = splitter.split_documents(article_docs)
    print(len(splits))
    
    # Process the first split
    extracted_content = []
    for split in splits:
        extracted_content.extend(extract(schema=schema, content=split.page_content))
    
    pprint.pprint(extracted_content)
    
    return extracted_content


urls = ["https://search.naver.com/search.naver?sm=tab_hty.top&where=news&ssc=tab.news.all&query=%EB%9D%BC%EC%9D%B4%EC%98%A8%EC%A6%88&oquery=%EC%82%BC%EC%84%B1&tqi=iCyRydpzL8VssNVAU%2FGssssstkd-519064"]
extracted_content = scrape_with_playwright(urls, schema=schema)

3245
[Document(page_content='연합뉴스 https://www.yna.co.kr/ 박진만 삼성 라이온즈 감독 https://www.yna.co.kr/view/PYH20240521184900007?input=1196m 박진만 삼성 라이온즈 감독이 21일 대구삼성라이온즈파크에서 열리는 2024 프로야구 kt wiz와 홈 경기를 앞두고 인터뷰하고 있다. 2024.5.21 https://www.yna.co.kr/view/PYH20240521184900007?input=1196m 경북매일신문 http://www.kbmaeil.com 소진공 대경본부 \'전통시장×라이온즈 홈런치장\' 행사 http://www.kbmaeil.com/news/articleView.html?idxno=997137 지난해 진행된 \'전통시장x스포츠\' 이벤트 행사에 삼성라이온즈 야구 경기를 보러온 관람객들이 참여하고 있는 모습 소상공인시장진흥공단(이하 소진공) 대구경북지역본부가 22일 대구삼성라이온즈파크에서 \'전통시장×라이온즈 홈런치장(場)\'행사를 개최한다. 이번 행사는 오는 28일까지 진행되는 5월... http://www.kbmaeil.com/news/articleView.html?idxno=997137 매일신문 https://www.imaeil.com/ \'원태인 아쉬운 투구\' 삼성 라이온즈, 김영웅 3점포에도 KT에 패배 https://www.imaeil.com/page/view/2024052118061452630 삼성 라이온즈가 21일 대구 삼성라이온즈파크에서 연장 접전 끝에 KT 위즈에 5대8로 패했다. 선발 등판한 에이스 원태인이 5이닝 7피안타 3실점에 그친 게 아쉬웠다. 경기 후반 김영웅이 3점 홈런을 터뜨려 승부를 연장으로 몰고 갔으나 끝내 웃지 못했다. 스물넷에 불과하지만 원태인은 이미 삼성의 에이스.... https://www.imaeil.com/page/view/2024052118061452630 대구일보 http://www.idaegu.com 상

#### LangSmith를 통한 모니티링
- [#18.LangSmith를 이용한 Langchain agent 내부 동작 구조 이해 - 조대협](https://bcho.tistory.com/1427)


<img src="./images/langsmith.png" width="800">